In [1]:
import pandas as pd
import numpy as np
import requests
from bs4 import BeautifulSoup
from tqdm import tqdm
import sys

# Скачиваем словарь с сайта

Скачаем список лексем с [заглавной страницы](https://gufo.me/dict/mikhelson)

In [ ]:
def get_lexems_one_letter (letter, n_page):
    try:
      r = requests.get('https://gufo.me/dict/mikhelson?page='+str(n_page)+'&letter='+letter)
      soup = BeautifulSoup(r.text, 'html.parser')
      res = soup.find(class_='header class="header hidden-print"')
      text = soup.find_all('li')
      return [t.text for t in text]
    except Exception:
      pass
  

In [ ]:
list_of_lexems = []
letters = ['а', 'б', 'в','г','д','е','ё','ж','з','и','й','к','л','м','н','о','п','р','с','т','у','ф','х','ц','ч','ш','щ','ъ','ы','ь','э','ю','я', 'o', 'p' ]

for letter in letters:
  
    for n_page in range(1, 15):
      try:
        list_of_lexems.extend(get_lexems_one_letter(letter, n_page))
      except Exception:
          pass

Удаляем все, что скачалось случайно

In [ ]:
site_info = ['Блог', 'Ежи Лец', 'Контакты', 'Пользовательское соглашение', '\nАлфавитный указатель\n', '\nПоиск по алфавитному указателю\n' ]
def del_site_info(list_of_lexems):
  return [lexem for lexem in list_of_lexems if lexem not in site_info]
list_of_lexems = del_site_info(list_of_lexems)


Количество лексем в словаре:

In [ ]:
len(list_of_lexems)

12418

In [ ]:
with open("list_of_lexems_Mihelson.txt", "w") as output:
    output.write(str(list_of_lexems))

In [ ]:
with open ('list_of_lexems_Mihelson.txt', 'r') as f:
  list_of_lexems = (f.read())
list_of_lexems = list_of_lexems[2:-3]
list_of_lexems = list_of_lexems.split('\', \'')

Генерируем список ссылок для всех лексем

In [ ]:
def generate_links(list_of_lexems, list_of_links):
  list_of_lexems = [lexem.replace(' -', '').replace('?', '').replace('!', '').replace(' ', '_') for lexem in list_of_lexems]
  list_of_links.extend( ['https://gufo.me/dict/mikhelson/'+word for word in list_of_lexems])
list_of_links = []
generate_links(list_of_lexems, list_of_links)

По сгенерированным ссылкам скачиваем словарь

In [ ]:
def split_author_and_book(source, lexem):
  source.replace('Гр. ', 'Гр.')
  source.replace('Иис. ', 'Иис.')
  source = source.split('. ')
  if len(source) == 1:
    author = source
    book = 'не указано'
  elif len(source[0])<=3:
    author = source[0]+'. '+source[1]
    try:
      book = source[2]
    except Exception:
      book = 'нет'
  else:
    author = source[0]
    book = source[1]
  try:
    comment = source[2:]
  except Exception:
    comment = 'нет'
  line = [lexem, author, book, comment]
  return line

In [ ]:
def get_site_data(link):
  r = requests.get(link)
  soup = BeautifulSoup(r.text, 'html.parser')
  text = soup.find_all('span', attrs={'style':'color:olive'})
  sources = [t.text for t in text]
  return sources

In [ ]:
table = []
with tqdm(total=4000, file=sys.stdout) as prbar:
  for link in list_of_links[8000:]:
    sources = get_site_data(link)
    for source in sources:
      print(split_author_and_book(source, link[31:]))
      # full_line = [link[32:]].extend(split_author_and_book(source))
      # table.append(full_line)
      table.append(split_author_and_book(source, link[32:]))
  prbar.update(1)
table

Выходные данные были обрезаны до нескольких последних строк (5000).
['святая_Русь', 'Жуковский', 'Русская слава.', []]
['святая_Русь', 'Марлинский', 'Поволжские разбойники.', []]
['святая_Святых', 'Евгений Марков', 'Труженик народной поэзии', ['("Нов', 'вр." 1900 г', '26 авг.)']]
['святая_Святых', 'Потапенко', 'Дочь курьера', ['1, 4.']]
['святая_Святых', 'П. Боборыкин', 'Перевал', ['Перевал', '3, 5.']]
['святая_Святых', 'Гр. Л.Н', 'Толстой', ['Толстой', 'Анна Каренина', '8, 1.']]
['святая_Святых', 'А.Г. В гостях у Айвазовского по поводу его 60-летнего юбилея 26-го сент', '1897 г', ['1897 г', '("Новости" 3 окт', '1897 г.)']]
['святая_Святых', 'Лесков', 'Воительница', ['4.']]
['святая_Святых', 'Гл. Успенский', 'Через пень-колоду', ['Через пень-колоду', '2, 3.']]
['святая_Святых', 'Маркевич', 'Марина из Алого Рога', ['7.']]
['святая_Святых', 'Маркевич', 'Четверть века назад', ['1, 5.']]
['святая_Святых', 'Б.М. Маркевич', 'Бездна', ['Бездна', 'Пролог', '5.']]
['святая_Святых', 'Там же', '3

[['оевши_(наевшись)_гороху_(говорить)',
  'Салтыков',
  'Сказки',
  ['Карась-идеалист.']],
 ['оевши_(наевшись)_гороху_(говорить)', 'Мельников', 'В лесах', ['3, 1.']],
 ['оел_казак_да_и_на_бок,_оттого_казак_и_гладок',
  'Лесков',
  'Старые годы в селе Плодомасове',
  ['3, 1.']],
 ['оется_там,_где_воля,_и_холя,_и_доля', 'Д.Д. Минаев.', 'нет', []],
 ['оется_там,_где_воля,_и_холя,_и_доля', ['Глинка.'], 'не указано', []],
 ['оется_там,_где_воля,_и_холя,_и_доля',
  'Кн. В.Ф',
  'Одоевский.',
  ['Одоевский.']],
 ['оется_там,_где_воля,_и_холя,_и_доля',
  'Лермонтов',
  'Герой нашего времени',
  ['Журнал Печорина', 'Тамань.']],
 ['оется_там,_где_воля,_и_холя,_и_доля', ['Екатерина II.'], 'не указано', []],
 ['оехал_туда_для_порядка,_а_возвратился_пьян',
  'Гоголь',
  'Ревизор',
  ['1, 5', 'Частный пристав.']],
 ['ожар_способствовал_к_украшенью',
  'Грибоедов',
  'Горе от ума',
  ['2, 5', 'Скалозуб о Москве.']],
 ['ожар', 'Грибоедов', 'Горе от ума', ['2, 5.']],
 ['ожарские_котлеты',
  'А.С. Пушки

In [ ]:
table = pd.DataFrame(table)
table.to_csv('Mihelson4.csv')

In [2]:
table1 = pd.read_csv('Mihelson4.csv')
table2 = pd.read_csv('Mihelson3.csv')
table3 = pd.read_csv('Mihelson.csv')
table4 = pd.read_csv('Mihelson1.csv')

In [8]:
table = pd.concat([table1, table2, table3, table4])

TypeError: ignored

In [34]:
new_indexes = [i for i in range(len(table))]
table['index'] = new_indexes

In [13]:
table.columns = ['index', 'lexem', 'author', 'book', 'comment']

В словаре иногда вместо указания автора пишут "там же". Исправим такие записи, заменив "там же" на авторов и произведения из предыдущих строк


In [35]:
table

,index,lexem,author,book,comment
0,0,оевши_(наевшись)_гороху_(говорить),Салтыков,Сказки,['Карась-идеалист.']
1,1,оевши_(наевшись)_гороху_(говорить),Мельников,В лесах,"['3, 1.']"
2,2,"оел_казак_да_и_на_бок,_оттого_казак_и_гладок",Лесков,Старые годы в селе Плодомасове,"['3, 1.']"
3,3,"оется_там,_где_воля,_и_холя,_и_доля",Д.Д. Минаев.,нет,[]
4,4,"оется_там,_где_воля,_и_холя,_и_доля",['Глинка.'],не указано,[]
...,...,...,...,...,...
6740,27727,одшофе,Достоевский,Скверный анекдот.,[]
6741,27728,одшофе,Писемский,Масоны,"['4, 9.']"
6742,27729,одшофе,Писемский,Люди сороковых годов,"['1, 18.']"
6743,27730,одымай_выше,Боборыкин,Василий Теркин,"['1, 25.']"


In [36]:
list_of_repeated_authors = list(table[table['author'] == 'Там же']['index'])

In [37]:
sorted(list_of_repeated_authors)

[7536,
 7543,
 7643,
 7736,
 7777,
 7800,
 7837,
 7861,
 7910,
 7965,
 7975,
 7977,
 7983,
 8062,
 8181,
 8194,
 8203,
 8207,
 8238,
 8645,
 8699,
 8714,
 8722,
 8823,
 8913,
 8932,
 8957,
 9210,
 9307,
 9325,
 9383,
 9432,
 9655,
 9711,
 9734,
 9851,
 9877,
 9878,
 9981,
 10020,
 10203,
 10204,
 10244,
 10245,
 10519,
 10993,
 11053,
 11161,
 11199,
 11223,
 11421,
 11422,
 11580,
 11613,
 11732,
 11791,
 11802,
 11964,
 12173,
 12194,
 12238,
 12283,
 12348,
 12356,
 12371,
 12379,
 12393,
 12425,
 12439,
 12711,
 13119,
 13139,
 13180,
 13208,
 13296,
 13300,
 13312,
 13355,
 13485,
 13742,
 13770,
 13808,
 14031,
 14126,
 14128,
 14237,
 14242,
 14374,
 14378,
 14437,
 14500,
 14525,
 14619,
 14620,
 14729,
 14843,
 14906,
 15059,
 15198,
 15210,
 15542,
 15616,
 15643,
 15734,
 15850,
 15851,
 15906,
 15963,
 16004,
 16102,
 16160,
 16544,
 16565,
 16705,
 16712,
 16772,
 16918,
 16931,
 16980,
 17113,
 17168,
 17187,
 17215,
 17354,
 17380,
 17704,
 17755,
 17817,
 17923,
 17924,

In [42]:
for index in (list_of_repeated_authors):
  table.loc[table['index'] == index, 'author'] = table.loc[table['index'] == index-1,'author']
  table.loc[table['index'] == index, 'book'] = table.loc[table['index'] == index-1,'book']